In [159]:
import ipytest
ipytest.autoconfig()

Read lines from the input:

In [160]:
%%ipytest -vv

def to_int(array):
  return [int(x) for x in array]

def read_lines(file_path):
  lines = []
  with open(file_path) as f:
    for line in f:
      line = line.rstrip('\n')
      coordinates = line.split(" -> ")
      start_coordinate = to_int(coordinates[0].split(','))
      end_coordinate = to_int(coordinates[1].split(','))
      lines.append((tuple(start_coordinate), tuple(end_coordinate)))
  return lines

def test_read_lines():
  file_path = "day5-test.input"
  lines = read_lines(file_path)
  assert lines == [((0,9), (5,9)),
                  ((8,0), (0,8)),
                  ((9,4), (3,4)),
                  ((2,2), (2,1)),
                  ((7,0), (7,4)),
                  ((6,4), (2,0)),
                  ((0,9), (2,9)),
                  ((3,4), (1,4)),
                  ((0,0), (8,8)),
                  ((5,5), (8,2))]

======================================= test session starts ========================================
platform darwin -- Python 3.9.4, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /Users/alpar/.pyenv/versions/3.9.4/bin/python
cachedir: .pytest_cache
rootdir: /Users/alpar/Documents/Projects/advent-of-code-2021/jupyter-notebook
plugins: anyio-3.4.0
collecting ... collected 1 item

tmpcfh1wmb3.py::test_read_lines PASSED                                                       [100%]

======================================== 1 passed in 0.01s =========================================


Consider only horizontal or vertical lines:

In [161]:
%%ipytest -vv

def keep_only_vertical_or_horizontal_lines(lines):
  return [line for line in lines if line[0][0] == line[1][0] or line[0][1] == line[1][1]]

def test_keep_only_vertical_or_horizontal_lines():
  file_path = "day5-test.input"
  lines = read_lines(file_path)
  filtered = keep_only_vertical_or_horizontal_lines(lines)
  assert filtered == [((0,9), (5,9)),
                  ((9,4), (3,4)),
                  ((2,2), (2,1)),
                  ((7,0), (7,4)),
                  ((0,9), (2,9)),
                  ((3,4), (1,4))]

======================================= test session starts ========================================
platform darwin -- Python 3.9.4, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /Users/alpar/.pyenv/versions/3.9.4/bin/python
cachedir: .pytest_cache
rootdir: /Users/alpar/Documents/Projects/advent-of-code-2021/jupyter-notebook
plugins: anyio-3.4.0
collecting ... collected 1 item

tmp7ey2z03v.py::test_keep_only_vertical_or_horizontal_lines PASSED                           [100%]

======================================== 1 passed in 0.02s =========================================


Determine diagram size:

In [162]:
%%ipytest -vv

def diagram_size(lines):
  return max([c for line in lines for coordinate in line for c in coordinate]) + 1

def test_diagram_size():
  file_path = "day5-test.input"
  lines = read_lines(file_path)
  size = diagram_size(lines)
  assert size == 10

======================================= test session starts ========================================
platform darwin -- Python 3.9.4, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /Users/alpar/.pyenv/versions/3.9.4/bin/python
cachedir: .pytest_cache
rootdir: /Users/alpar/Documents/Projects/advent-of-code-2021/jupyter-notebook
plugins: anyio-3.4.0
collecting ... collected 1 item

tmpxso0xyn6.py::test_diagram_size PASSED                                                     [100%]

======================================== 1 passed in 0.01s =========================================


Draw vertical and horizontal lines on diagram:

In [163]:
%%ipytest -vv

def draw_point_on_diagram(diagram, diagram_size, coordinate):
  idx = diagram_size * coordinate[1] + coordinate[0]
  if diagram[idx] == '.':
    diagram[idx] = 1
  else:
    diagram[idx] = diagram[idx] + 1

def draw_line_on_diagram(diagram, diagram_size, line):
  if line[0][0] == line[1][0]:
    for idx in range_on_axis(line[0][1], line[1][1]):
      draw_point_on_diagram(diagram, diagram_size, (line[0][0], idx))
  elif line[0][1] == line[1][1]:
    for idx in range_on_axis(line[0][0], line[1][0]):
      draw_point_on_diagram(diagram, diagram_size, (idx, line[0][1]))
  else:
    for x in range_on_axis(line[0][0], line[1][0]):
      y = y_for_x_on_line(x, line)
      draw_point_on_diagram(diagram, diagram_size, (x, y))

def range_on_axis(start, end):
  if start < end:
    return range(start, end + 1)
  else:
    return range(start, end - 1, -1)

def y_for_x_on_line(x, line):
  y = (((float(x) - float(line[0][0])) / (float(line[1][0]) - float(line[0][0]))) * (float(line[1][1]) - float(line[0][1]))) + float(line[0][1])
  # print((x, y))
  # print((x, round(y)))
  return round(y)

"""
(y - y1) / (y2 - y1) = (x - x1) / (x2 - x1)
(y - y1) = ((x - x1) / (x2 - x1)) * (y2 - y1)
y = (((x - x1) / (x2 - x1)) * (y2 - y1)) + y1
"""

def create_diagram(lines):
  size = diagram_size(lines)
  return (['.' for _ in range(size) for _ in range(size)], size)

def draw_lines_on_diagram(lines):
  (diagram, size) = create_diagram(lines)
  for line in lines:
    draw_line_on_diagram(diagram, size, line)
  return (diagram, size)

def print_diagram(diagram, diagram_size):
  output = ""
  for (idx, value) in enumerate(diagram):
    output += str(value)
    if idx % diagram_size == diagram_size-1:
      output += "\n"
  print(output)

def test_draw_lines_on_diagram():
  file_path = "day5-test.input"
  lines = read_lines(file_path)
  filtered = keep_only_vertical_or_horizontal_lines(lines)
  (diagram, _) = draw_lines_on_diagram(filtered)
  assert diagram == ['.','.','.','.','.','.','.', 1 ,'.','.',
'.','.', 1 ,'.','.','.','.', 1 ,'.','.',
'.','.', 1 ,'.','.','.','.', 1 ,'.','.',
'.','.','.','.','.','.','.', 1 ,'.','.',
'.', 1 , 1 , 2 , 1 , 1 , 1 , 2 , 1 , 1 ,
'.','.','.','.','.','.','.','.','.','.',
'.','.','.','.','.','.','.','.','.','.',
'.','.','.','.','.','.','.','.','.','.',
'.','.','.','.','.','.','.','.','.','.',
 2 , 2 , 2 , 1 , 1 , 1 ,'.','.','.','.']


======================================= test session starts ========================================
platform darwin -- Python 3.9.4, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /Users/alpar/.pyenv/versions/3.9.4/bin/python
cachedir: .pytest_cache
rootdir: /Users/alpar/Documents/Projects/advent-of-code-2021/jupyter-notebook
plugins: anyio-3.4.0
collecting ... collected 1 item

tmppxjpkqmv.py::test_draw_lines_on_diagram PASSED                                            [100%]

======================================== 1 passed in 0.01s =========================================


Number of points where at least two lines overlap:

In [164]:
%%ipytest -vv

from functools import reduce

def number_of_points_where_at_least_two_lines_overlap(diagram):
  return reduce(lambda counter,point: counter+1 if point != '.' and point >= 2 else counter, diagram, 0)

def test_number_of_points_where_at_least_two_lines_overlap():
  file_path = "day5-test.input"
  lines = read_lines(file_path)
  filtered = keep_only_vertical_or_horizontal_lines(lines)
  (diagram, _) = draw_lines_on_diagram(filtered)
  result = number_of_points_where_at_least_two_lines_overlap(diagram)
  assert result == 5



======================================= test session starts ========================================
platform darwin -- Python 3.9.4, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /Users/alpar/.pyenv/versions/3.9.4/bin/python
cachedir: .pytest_cache
rootdir: /Users/alpar/Documents/Projects/advent-of-code-2021/jupyter-notebook
plugins: anyio-3.4.0
collecting ... collected 1 item

tmp7s18mq4b.py::test_number_of_points_where_at_least_two_lines_overlap PASSED                [100%]

======================================== 1 passed in 0.02s =========================================


Solution part 1:

In [165]:
file_path = "day5.input"
lines = read_lines(file_path)
filtered = keep_only_vertical_or_horizontal_lines(lines)
(diagram, _) = draw_lines_on_diagram(filtered)
result = number_of_points_where_at_least_two_lines_overlap(diagram)
print(result)

6564


Draw diagonal lines as well:

In [166]:
%%ipytest -vv

def test_draw_lines_on_diagram():
  file_path = "day5-test.input"
  lines = read_lines(file_path)
  (diagram, size) = draw_lines_on_diagram(lines)
  print_diagram(diagram, size)
  assert diagram == [1 ,'.', 1 ,'.','.','.','.', 1 , 1 ,'.',
'.', 1 , 1 , 1 ,'.','.','.', 2 ,'.','.',
'.','.', 2 ,'.', 1 ,'.', 1 , 1 , 1 ,'.',
'.','.','.', 1 ,'.', 2 ,'.', 2 ,'.','.',
'.', 1 , 1 , 2 , 3 , 1 , 3 , 2 , 1 , 1 ,
'.','.','.', 1 ,'.', 2 ,'.','.','.','.',
'.','.', 1 ,'.','.','.', 1 ,'.','.','.',
'.', 1 ,'.','.','.','.','.', 1 ,'.','.',
 1 ,'.','.','.','.','.','.','.', 1 ,'.',
 2 , 2 , 2 , 1 , 1 , 1 ,'.','.','.','.']

======================================= test session starts ========================================
platform darwin -- Python 3.9.4, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /Users/alpar/.pyenv/versions/3.9.4/bin/python
cachedir: .pytest_cache
rootdir: /Users/alpar/Documents/Projects/advent-of-code-2021/jupyter-notebook
plugins: anyio-3.4.0
collecting ... collected 1 item

tmpveeqeyuf.py::test_draw_lines_on_diagram PASSED                                            [100%]

======================================== 1 passed in 0.02s =========================================


Number of points where at least two lines overlap including diagonal lines:

In [167]:
%%ipytest -vv

def test_number_of_points_where_at_least_two_lines_overlap():
  file_path = "day5-test.input"
  lines = read_lines(file_path)
  (diagram, _) = draw_lines_on_diagram(lines)
  result = number_of_points_where_at_least_two_lines_overlap(diagram)
  assert result == 12

======================================= test session starts ========================================
platform darwin -- Python 3.9.4, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /Users/alpar/.pyenv/versions/3.9.4/bin/python
cachedir: .pytest_cache
rootdir: /Users/alpar/Documents/Projects/advent-of-code-2021/jupyter-notebook
plugins: anyio-3.4.0
collecting ... collected 1 item

tmp2aa6andy.py::test_number_of_points_where_at_least_two_lines_overlap PASSED                [100%]

======================================== 1 passed in 0.01s =========================================


Solution part 2:

In [168]:
file_path = "day5.input"
lines = read_lines(file_path)
(diagram, size) = draw_lines_on_diagram(lines)
result = number_of_points_where_at_least_two_lines_overlap(diagram)
print(result)

19172
